# Python solve Maze!!!

* Solving maze without machine learning, infact no larning :P

* A simple image processing approach to solve maze using morphological operations

reference: [Guide to Signals and Patterns in Image Processing](https://link.springer.com/book/10.1007/978-3-319-14172-5)


In [1]:
# download and install all the requirements
# set autoreload 2 to reload .py script changes
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
# conver the image to binary
# crop maze from image
# get the thickness of maze path in pixel
def preProcessMaze(oimg):
    img = cv2.cvtColor(oimg, cv2.COLOR_BGR2GRAY)
    # create binary image using otsu adaptive thresholding
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    # trim the image border
    r,c = np.where(img==255)
    x0,x1, y0, y1 = r[0], r[-1], c[0], c[-1]
    img = img[y0:y1, x0:x1] # crop roi
    # on four sides search no of zero pixel on border
    mazePathSize = max([(img[0] == 0).sum(),(img[:,0] == 0).sum(),
                            (img[1] == 0).sum(), (img[:, 1] == 0).sum()])
    return img, mazePathSize

Hints:
*   A maze consists of walls (255) and paths (0)
*   Where the correct path split the maze into two parts.
* Dead ends are basically paths (0) surrounded by wall (255) can be detected using closing operations

Algorithm:
1. Select any one contour and choose a ksize (for good result choose kernel size == pathSize).
2. Dilate the contour. This will generate a mask with all possible paths
3. Perform closing opertion on the image. This mask shows all the dead ends
4. Now find the final path by removing dead ends from the all possible paths.


In [3]:
def solveMaze(img, kSize, contourIx):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contourImg = cv2.drawContours(np.zeros_like(img), contours, contourIx, 255, 5)
    kernel = np.ones((kSize, kSize), 'u1')
    # to solve maze subtract dilation with close
    dilation = cv2.morphologyEx(contourImg, cv2.MORPH_DILATE, kernel)
    close = cv2.morphologyEx(dilation, cv2.MORPH_ERODE, kernel) # perform closing
    diff = cv2.bitwise_xor(dilation, close) # bitwise subraction
    return diff, contourImg, dilation, close

In [18]:
def main(imPath, contourIx, enablePathSize, pathSize, figsize):
    fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(figsize, figsize))
    oimg = cv2.imread(imPath)
    dispImgs = []
    img, mazePathSize = preProcessMaze(oimg)
    if enablePathSize:
        mazePathSize = pathSize 
    path, contourImg, dilation , close = solveMaze(img, mazePathSize, contourIx)
    dispImgs.append(["Contour Image", contourImg.copy()])
    dispImgs.append(["Dilated Image", dilation.copy()])
    dispImgs.append(["Close Image", close.copy()])
    dispImgs.append(["Input", oimg])
    dispImgs.append(["Path", path.copy()])
    img[path == 255] = 196
    dispImgs.append(["Solution [size: %s]" % mazePathSize, img.copy()])
    for ax, (title, img) in zip(axs.flat, dispImgs):
        ax.imshow(img, interpolation='bilinear')
        ax.set_title(title)
        ax.axis('off')
    plt.show()
print("Before using track bar please enablePathSize")
a = interact(main, imPath=glob('maze*.*'),contourIx=[0,1], enablePathSize=False, pathSize=(1,100), figsize=(5, 16))

Before using track bar please enablePathSize


interactive(children=(Dropdown(description='imPath', options=('maze3.png', 'maze1.png', 'maze2.png'), value='m…